In [64]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import scipy
from scipy.stats import wilcoxon
import datetime

Get total amount spent in each state for the 2024 election. 

In [65]:
years = ['2008', '2012', '2016', '2020', '2024']
spending_dfs = dict()
state_sums = dict()
for year in years:
    spending_dfs[year] = pd.read_csv(f"spending_data_{year}.csv", index_col=False, low_memory=False)
    # state_sums[year] = spending_dfs[year].groupby('recipient_st').sum('disb_amt').sort_values('disb_amt')

# spending_24_df = pd.read_csv("spending_data_2024.csv", index_col=False, low_memory=False)
# spending_24_df.groupby('recipient_st').sum('disb_amt').sort_values('disb_amt')
# spending_24_df.head()
# spending_dfs
# state_sums['2024']

In [66]:
for year, df in spending_dfs.items():
    display(year, df['recipient_st'].unique())
    # display(df.groupby('recipient_st').count())

# display(spending_24_df['recipient_st'].unique())
states = ['AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'FL', 'GA', 'IA', 'IL',
       'IN', 'LA', 'MA', 'MD', 'ME', 'MN', 'MO', 'NC', 'NE', 'NH', 'NJ',
       'NY', 'OK', 'PA', 'SC', 'TN', 'TX', 'VA', 'WA', 'WI', 'WY',
       'OH', 'WV', 'AK', 'DE', 'HI', 'ID', 'KS', 'KY', 'MI',
       'MS', 'MT', 'ND', 'NM', 'NV', 'OR', 'RI', 'UT', 'VT',
       'SD', 'C', 'F', 'G', 'I', 'N',
       'T', 'AA'] 
# C F G I N T AA are errors that need to be replaced with the correct state 

'2008'

array(['AL', 'AZ', 'CA', 'DC', 'DE', 'FL', 'GA', 'IA', 'ID', 'IL', 'MA',
       'MD', 'MO', 'NC', 'NE', 'NH', 'NJ', 'NY', 'OH', 'OK', 'SC', 'SD',
       'TN', 'TX', 'VA', 'WA', 'WV', 'AK', 'AR', 'CO', 'CT', 'IN', 'KY',
       'LA', 'ME', 'MI', 'MN', 'NV', 'PA', 'UT', '99', 'FF', 'GU', 'HI',
       'KS', 'MS', 'MT', 'ND', 'NM', 'NS', 'ON', 'OR', 'PR', 'QC', 'RI',
       'UK', 'VT', 'WI', 'WY', 'XX', 'Z', 'ZZ', 'AA', 'AB', 'BC', 'CH',
       'JA', 'KL', 'MB', 'SO', '10', '13', '60', 'AS', 'BE', 'C', 'CN',
       'EN', 'FR', 'GE', 'GR', 'I', 'II', 'IS', 'JO', 'JR', 'K', 'KA',
       'LU', 'N', 'RP', 'SH', 'SW', 'TW', 'VI', 'VW', '46', 'IR', 'MY',
       'OA', 'PU', 'T', 'WW'], dtype=object)

'2012'

array(['AZ', 'CA', 'CO', 'DC', 'FL', 'GA', 'IA', 'IL', 'KS', 'LA', 'MD',
       'MN', 'NE', 'NH', 'NJ', 'NY', 'OH', 'PA', 'SC', 'TX', 'UT', 'VA',
       'WA', 'WI', 'AK', 'AL', 'AR', 'BC', 'CT', 'D.', 'DE', 'HI', 'ID',
       'IN', 'KY', 'MA', 'ME', 'MI', 'MO', 'MS', 'MT', 'MY', 'NC', 'ND',
       'NM', 'NV', 'OK', 'OR', 'PO', 'PR', 'RI', 'SD', 'SW', 'TN', 'VT',
       'WV', 'WY', 'HA', 'JO', 'MH', 'NS', 'ON', 'QC', 'ZZ', 'GU', 'AU',
       'N.', 'HN'], dtype=object)

'2016'

array(['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
       'HI', 'IA', 'ID', 'IL', 'IN', 'KA', 'KS', 'KY', 'LA', 'MA', 'MD',
       'ME', 'MI', 'MN', 'MO', 'MP', 'MT', 'NB', 'NC', 'NH', 'NJ', 'NV',
       'NY', 'OH', 'OK', 'ON', 'PA', 'PR', 'RI', 'SA', 'SC', 'SD', 'TN',
       'TX', 'UK', 'UT', 'VA', 'VI', 'VT', 'WA', 'WI', 'WV', 'WY', 'AB',
       'AM', 'BC', 'NE', 'OR', 'ZZ', 'AU', 'GB', 'GU', 'MS', 'ND', 'NM',
       'QC', 'D.', 'LO', 'AS', 'FF'], dtype=object)

'2020'

array(['AZ', 'CA', 'CO', 'CT', 'DC', 'FL', 'GA', 'IA', 'IL', 'LA', 'MD',
       'MN', 'MO', 'NC', 'NE', 'NH', 'NJ', 'NV', 'NY', 'OH', 'OK', 'PA',
       'SC', 'TN', 'TX', 'UK', 'WA', 'AL', 'AR', 'BC', 'DE', 'HI', 'ID',
       'IN', 'KY', 'MA', 'ME', 'MI', 'ND', 'OR', 'SD', 'UT', 'VA', 'WI',
       'WV', 'WY', 'VT', 'AK', 'KS', 'MP', 'MS', 'MT', 'NM', 'RI', 'ZZ',
       'AB', 'EA', 'ON', 'PE', 'ST', 'PR', 'AS', 'NS', 'AU', 'VI', 'SU',
       'BR', 'AA', 'AE'], dtype=object)

'2024'

array(['AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'FL', 'GA', 'IA', 'IL',
       'IN', 'LA', 'MA', 'MD', 'ME', 'MN', 'MO', 'NC', 'NE', 'NH', 'NJ',
       'NY', 'OK', 'PA', 'SC', 'TN', 'TX', 'VA', 'VI', 'WA', 'WI', 'WY',
       'ZZ', 'OH', 'WV', 'AK', 'AS', 'DE', 'HI', 'ID', 'KS', 'KY', 'MI',
       'MP', 'MS', 'MT', 'ND', 'NM', 'NV', 'OR', 'RI', 'UT', 'VT', 'GU',
       'PR', 'SD', 'AB', 'GB', 'ON', 'AU', 'C', 'F', 'G', 'I', 'N', 'QC',
       'T', 'TH', 'UK', 'W', 'AA'], dtype=object)

Upon inspection some states have been labeled wrong. For example some rows have C, since they occur in san francisco we can assume this should be CA. 

In [79]:
# correct 2008 states 
spending_dfs['2008']['recipient_st']= spending_dfs['2008']['recipient_st'].replace({'AA': 'MA', 'C': 'CA', 'I': 'IA', 'II': 'IL', 'K': 'KS', 'KA': 'KS' , 'N': 'NC', 'VW': 'WV', '46': 'IN', 'MY': 'MT', 'OA': 'PA', 'T': 'TX', 'WW': 'WA'})

# AA = MA
# C = CA
# I = Iowa
# ii = illinois
# k = kansas
# ka = kansas 
# N = nc
# vw = wv
# 46 = indianapolis
# MY = MT
# OA = pa
# t = tx
# ww = wa


# correct 2012 states 
spending_dfs['2012']['recipient_st']= spending_dfs['2012']['recipient_st'].replace({'D.': 'DC', 'MY': 'NY', 'HA': 'HI', 'MH': 'NH', 'HN': 'NH'})

# "D." = "DC"
# MY = NY
# HA = HI
# MH = NH
# HN = HN

# correct 2016 states 
spending_dfs['2016']['recipient_st']= spending_dfs['2016']['recipient_st'].replace({'D.': 'DC', 'MY': 'NY', 'HA': 'HI', 'MH': 'NH', 'HN': 'NH'})

# nb = ne
# d. = dc


# correct 2020 states 
# none


# correct 2024 states 
spending_dfs['2024']['recipient_st']= spending_dfs['2024']['recipient_st'].replace({'C': 'CA', 'AA': 'CA', 'F': 'FL', 'G': 'GA', 'T': 'TX'})
spending_dfs['2024'].loc[spending_dfs['2024']['recipient_zip'] == '46038', 'recipient_st'] = 'IN'
spending_dfs['2024']['recipient_st'] = spending_dfs['2024']['recipient_st'].replace('I', 'IA')
spending_dfs['2024'].loc[spending_dfs['2024']['recipient_zip'] == '03276', 'recipient_st'] = 'NH'
spending_dfs['2024'].loc[spending_dfs['2024']['recipient_zip'] == '03063', 'recipient_st'] = 'NH'
spending_dfs['2024']['recipient_st'] = spending_dfs['2024']['recipient_st'].replace('N', 'NY')


# display(spending_dfs['2024']['recipient_st'].unique())


# display(spending_dfs['2024'][spending_dfs['2024']['recipient_st'] == "N"])
# I needs to be handled separatley, 3 are Iowa 1 is indiana, so does N
# display(spending_dfs['2024']['recipient_st'].unique())
# states = ['AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'FL', 'GA', 'IA', 'IL',
#        'IN', 'LA', 'MA', 'MD', 'ME', 'MN', 'MO', 'NC', 'NE', 'NH', 'NJ',
#        'NY', 'OK', 'PA', 'SC', 'TN', 'TX', 'VA', 'WA', 'WI', 'WY',
#        'OH', 'WV', 'AK', 'DE', 'HI', 'ID', 'KS', 'KY', 'MI',
#        'MS', 'MT', 'ND', 'NM', 'NV', 'OR', 'RI', 'UT', 'VT',
#        'SD', 'C', 'F', 'G', 'I', 'N',
#        'T', 'AA'] 

# spending_dfs['2024']['recipient_st'].unique()
# spending_dfs['2024'][spending_dfs['2024']['recipient_st'].isin(['MD', 'NC'])]

states = [
        'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'FL', 'GA', 'IA', 
        'IL', 'IN', 'LA', 'MA', 'MD', 'ME', 'MN', 'MO', 'NC', 'NE', 
        'NH', 'NJ', 'NY', 'OK', 'PA', 'SC', 'TN', 'TX', 'VA', 'SD',
        'WA', 'WI', 'WY', 'OH', 'WV', 'AK', 'DE', 'HI', 'ID', 'KS', 
        'KY', 'MI', 'MS', 'MT', 'ND', 'NM', 'NV', 'OR', 'RI', 'UT', 
        'VT'
        ]

# print(spending_dfs['2024']['recipient_st'].unique())
# spending_dfs['2024'] = spending_dfs['2024'][spending_dfs['2024']['recipient_st'].isin(states)]
# spending_dfs['2024'] = spending_dfs['2024'][spending_dfs['2024']['recipient_st'].isin(states)]
# spending_dfs['2024'] = spending_dfs['2024'][spending_dfs['2024']['recipient_st'].isin(states)]
# spending_dfs['2024'] = spending_dfs['2024'][spending_dfs['2024']['recipient_st'].isin(states)]
# spending_dfs['2024'] = spending_dfs['2024'][spending_dfs['2024']['recipient_st'].isin(states)]

# print(len(spending_dfs['2024']['recipient_st'].unique()))
for year in spending_dfs:
    spending_dfs[year] = spending_dfs[year][spending_dfs[year]['recipient_st'].isin(states)]
    # print(len(df['recipient_st'].unique()))

for df in spending_dfs.values():
    # df = df[df['recipient_st'].isin(states)]
    print(len(df['recipient_st'].unique()))
    
# spending_dfs['2024']['recipient_st'].unique()

90
63
70
69
51


In [78]:
# spending_dfs["2024"]['new_col'] = spending_dfs["2024"]['recipient_st'] + "_2024"
# spending_dfs["2024"]

for year in years:
    print(len(spending_dfs[year]['recipient_st'].unique()))
    state_sums[year] = spending_dfs[year].groupby('recipient_st').sum('disb_amt').sort_values('disb_amt')


for year, st_sum in state_sums.items():
    st_sum['state_year'] = st_sum.index + f"_{year}"


# state_sums['2024']
# spending_dfs["2024"]['recipient_st']

90
63
70
69
51


In [77]:
all_state_sums_df = pd.concat(state_sums.values())
all_state_sums_df

,disb_amt,file_num,state_year
recipient_st,,,
QC,5.063000e+01,876920,QC_2008
LU,8.753000e+01,10159293,LU_2008
RP,2.321000e+02,753674,RP_2008
AS,2.500000e+02,753674,AS_2008
CN,3.450000e+02,753817,CN_2008
...,...,...,...
FL,5.477338e+07,6992813013,FL_2024
CA,6.363917e+07,39235915976,CA_2024
VA,1.432239e+08,15642195573,VA_2024
